In [ ]:
!pip install mxnet

# Pretrained model

Hier verwenden wir ein bereits trainiertes, komplexes Convolutional Neural Network.

Das Netzwerk wurde auf dem CIFAR-10 Datensatz trainiert.

Es geht hier mehrheitlich darum, aufzuzeigen, wozu grosse CNNs bei **grossen Bilderdatensätzen** in der Lage sind.

In [ ]:
import numpy as np

from sklearn.metrics import accuracy_score

import pandas as pd
import pickle

In [13]:
!pip install tensorflow
!pip install gluoncv

You should consider upgrading via the '/Users/beni/Documents/Kennwerte/IdeaProjects/cas_machine_learning_cifar10/venv/bin/python -m pip install --upgrade pip' command.
You should consider upgrading via the '/Users/beni/Documents/Kennwerte/IdeaProjects/cas_machine_learning_cifar10/venv/bin/python -m pip install --upgrade pip' command.


In [14]:
# Class mapping from pretrained model
class_names = {
    0:'plane',
    1:'car',
    2:'bird',
    3:'cat',
    4:'deer',
    5:'dog',
    6:'frog',
    7:'horse',
    8:'ship',
    9:'truck'
}

## ResNet110 - Use pretrained model (on CIFAR 10)

Wir laden das bereits trainierte Netzwerk mittels der `mxnet` Library.

Wir müssen das Modell **nicht mehr trainieren** und laden es hier nur.

Das verwendete Modell ist eine Implementierung von diesem Paper: https://arxiv.org/abs/1603.05027

In [15]:
from mxnet.gluon.data.vision import transforms

import mxnet as mx
from gluoncv.model_zoo import get_model

ctx = [mx.cpu(0)]

# Get the model CIFAR_ResNet20_v1, with 10 output classes, without pre-trained weights
net = get_model('cifar_resnet110_v2', pretrained=True, classes=10)

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize([0.4914, 0.4822, 0.4465], [0.2023, 0.1994, 0.2010])  # Normalization ist analog wie bei Training des Netwerkes
])

# Predict classes for test set

If we are happy with the performance of our model on the validation set, we can apply it to the test set.

In [16]:
with open('../data/test.pkl', 'rb') as f:
    X_test = pickle.load(f)

In [17]:
%time

y_test_pred_prob = net(transform_test(mx.ndarray.array(X_test)))

y_test_pred = np.argmax(y_test_pred_prob.asnumpy(), axis=1)
# Decode numbers back to labels (order given by pretrained network)
y_test_pred_enc = np.vectorize(class_names.get)(y_test_pred)

y_test_pred_df = pd.DataFrame(y_test_pred_enc, columns=['label'])

CPU times: user 2 µs, sys: 1e+03 ns, total: 3 µs
Wall time: 5.25 µs


To submit the predictions to Kaggle we write them into a .csv file, which you can manually submit.

In [18]:
y_test_pred_df.to_csv('../out/cnn_pretrained.csv', header=True, index_label='id')